In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from time import perf_counter
from wordcloud import WordCloud, STOPWORDS
import sys, email, string
from nltk.stem import WordNetLemmatizer
import nltk
import os, gc, re
from IPython.display import Markdown, display
def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv')

In [3]:
df.head(10)

In [4]:
from nltk.tokenize import RegexpTokenizer

def clean_str(string, reg = RegexpTokenizer(r'[a-z]+')):
    string = string.lower()
    tokens = reg.tokenize(string)
    return " ".join(tokens)

In [5]:
df['text'] = df['text'].apply(lambda string: clean_str(string))

In [6]:
def rem_sub(i):
    return i.replace('subject', '')
df['text'] = df['text'].map(rem_sub)

In [7]:
df.head()
df.shape

In [8]:
x = df['text']
Y = df.label_num

In [9]:
print(x[0])
print(x.shape)

In [10]:
spam=[]
ham=[]
def split():
    for i in range(0,len(x)):
        if(df['label_num'][i]==0):
            ham.append(x[i])
        else:
            spam.append(x[i])
split()
print(ham[0])

In [11]:

stopwords = set(STOPWORDS)
print('Short Text Classification: A Survey')
#print(Counter(word_count(str(spam))).most_common(100))
for i in range (0,len(x)):
    text=' '.join(spam)
    
cloud = WordCloud(
    max_font_size=50,
    background_color='white',
    max_words=50
    ).generate(text)
fig = plt.figure(1)
plt.imshow(cloud)
plt.axis('off')
plt.show()
fig.savefig("Short_Text_Classification")

In [12]:
stopwords = set(STOPWORDS)
print('Short Text Classification: A Survey')
#print(Counter(word_count(str(spam))).most_common(100))
for i in range (0,len(x)):
    text1=' '.join(ham)
    
cloud = WordCloud(
    max_font_size=50,
    background_color='white',
    max_words=50
    ).generate(text1)
fig = plt.figure(1)
plt.imshow(cloud)
plt.axis('off')
plt.show()
fig.savefig("Short_Text_Classification")

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(x)

# Get the categories
y = df.label


In [14]:
print(x[0])

In [15]:
print(x.shape)

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test , y_train, y_test = train_test_split(x,y , test_size = 0.2, random_state =20)


In [17]:
print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

In [18]:
Accuracy=[]
Duration=[]
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
start = perf_counter()
logreg=LogisticRegression(solver='liblinear')
logreg=logreg.fit(x_train,y_train)
prediction=logreg.predict(x_test)
Accuracy.append(accuracy_score(prediction,y_test))
print(Accuracy[0])

duration = perf_counter() - start
duration = round(duration,2)
Duration.append(duration)
print(Duration[0])

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
start = perf_counter()
clf = MultinomialNB()
clf.fit(x_train, y_train)
naive_predict=clf.predict(x_test)
Accuracy.append(accuracy_score(naive_predict,y_test))
print(Accuracy[1])
duration = perf_counter() - start
duration = round(duration,2)
Duration.append(duration)
print(Duration[1])

In [20]:
from sklearn.ensemble import RandomForestClassifier
start = perf_counter()
forest_clf = RandomForestClassifier(max_depth=10, random_state=0)
forest_clf.fit(x_train,y_train)
ran_pred=forest_clf.predict(x_test)
Accuracy.append(accuracy_score(ran_pred,y_test))
print(Accuracy[2])
duration = perf_counter() - start
duration = round(duration,2)
Duration.append(duration)
print(Duration[2])

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
start = perf_counter()
tree = DecisionTreeClassifier(criterion='entropy') # function to measure the quality of split 
tree.fit(x_train,y_train)
ran=tree.predict(x_test)
Accuracy.append(accuracy_score(ran,y_test))
print(Accuracy[3])
duration = perf_counter() - start
duration = round(duration,2)
Duration.append(duration)
print(Duration[3])

In [22]:
model=["LogisticRegression", "MultinomialNB","RandomForestClassifier", "DecisionTreeClassifier"]
models_acc = []
for i in range(0,4):
    models_acc.append([model[i],Accuracy[i],Duration[i]])
df_acc = pd.DataFrame(models_acc)
df_acc.columns = ['Model', 'Accuracy', 'Training time (sec)']
df_acc.sort_values(by = 'Accuracy', ascending = False, inplace=True)
df_acc.reset_index(drop = True, inplace=True)
df_acc

In [23]:
plt.figure(figsize = (15,5))
sns.barplot(x = 'Model', y = 'Accuracy', data = df_acc)
plt.title('Accuracy on the test set\n(the Y-Axis is between 0.8 and 1.0)', fontsize = 15)
plt.ylim(0.7,1)
plt.show()

In [24]:
plt.figure(figsize = (15,5))
sns.barplot(x = 'Model', y = 'Training time (sec)', data = df_acc)
plt.title('Training time for each model in sec', fontsize = 15)
plt.ylim(0,1.5)
plt.show()

In [25]:
model = MultinomialNB()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
def display_result(df, number=2):
    for i in range(number):
        msg = df['text'].iloc[i]
        label = df["label"].iloc[i]
        msg_vec = cv.transform([msg])
        pred_label = model.predict(msg_vec)
        printmd(f"**Real: {label}, Predicted: {pred_label[0]}**")
        printmd(f"**E-Mail: {msg}**")
        printmd("_______________________________________________________________")
        print()
        print()
        
    
df_spam = df[df['label'] == 'spam']
df_ham = df[df['label'] == 'ham']
display_result(df_spam)
display_result(df_ham)